In [ ]:
!pip install Sastrawi
!pip install Keras-Preprocessing

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from nltk.tokenize import sent_tokenize, word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df1 = 'https://raw.githubusercontent.com/Zaniiiii/DataGemas/main/Baru/cleaning_result%20v1.csv'
df1 = pd.read_csv(df1, sep=",")
df = df1
df.head()

,full_text,text_clean,class
0,padahal nih yah ga usah pada banyak cingcong i...,perlu cingcong orang surabaya bandung medan ni...,0
1,Bisa naik jaklingko dari cipayung ke tamini 0 ...,naik jaklingko cipayung tamini rupiah wkwkwk l...,1
2,gw banget lagi terus naik jaklingko gratis ke ...,banget terus naik jaklingko gratis pelni,1
3,Ya mesti dibuat Stasiun baru karna Stasiun ter...,mesti buat stasiun baru stasiun dekat ancol ha...,2
4,Nilai tauladan yg bisa diambil dari nunggu jak...,nilai tauladan ambil nunggu jaklingko gin yaki...,0


In [ ]:
value_counts = df['class'].value_counts().min()

print(value_counts)

167


In [ ]:
# banyak = 0
# value_counts = df['class'].value_counts().min()
# hapus0 = []
# hapus1 = []

# for i in range(0,len(df)):
#   row = df.iloc[i]
#   if row['class'] == 0:
#     banyak = banyak + 1
#   if banyak >= value_counts:
#     if row['class'] == 0:
#       hapus1.append(i)


# df = df.drop(index=hapus0).reset_index(drop=True)

# banyak = 0

# for i in range(0,len(df)):
#   row = df.iloc[i]
#   if row['class'] == 2:
#     banyak = banyak + 1
#   if banyak >= value_counts:
#     if row['class'] == 2:
#       hapus1.append(i)

# df = df.drop(index=hapus1).reset_index(drop=True)

In [ ]:
# value_counts = df['class'].value_counts()

# print(value_counts)

In [ ]:
def pre_process(text):
    # Case Folding: Lowercase
    # Merubah format teks menjadi format huruf kecil semua (lowercase).
    text = text.lower()

    # Case Folding: Removing Number
    # Menghapus karakter angka.
    text = re.sub(r"\d+", "", text)

    # Case Folding: Removing Punctuation
    # Menghapus karakter tanda baca.
    text = text.translate(str.maketrans("","",string.punctuation))

    #Case Folding: Removing Whitespace
    #Menghapus karakter kosong.
    text = text.strip()


    #Separating Sentences with Split () Method
    #Fungsi split() memisahkan string ke dalam list dengan spasi sebagai pemisah jika tidak ditentukan pemisahnya.
    pisah = text.split()

    #Filtering using sastrawi
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    text =  stopword.remove(text)

    return text

df['text_clean'] = df['text_clean'].apply(lambda x:pre_process(x))
df.head()

,full_text,text_clean,class
0,padahal nih yah ga usah pada banyak cingcong i...,perlu cingcong orang surabaya bandung medan ni...,0
1,Bisa naik jaklingko dari cipayung ke tamini 0 ...,naik jaklingko cipayung tamini rupiah wkwkwk l...,1
2,gw banget lagi terus naik jaklingko gratis ke ...,banget terus naik jaklingko gratis pelni,1
3,Ya mesti dibuat Stasiun baru karna Stasiun ter...,mesti buat stasiun baru stasiun dekat ancol ha...,2
4,Nilai tauladan yg bisa diambil dari nunggu jak...,nilai tauladan ambil nunggu jaklingko gin yaki...,0


In [ ]:
train_df, test_df = train_test_split(df, test_size = 0.1, random_state = 42)
print("Training data size : ", train_df.shape)
print("Test data size : ", test_df.shape)

Training data size :  (1080, 3)
Test data size :  (120, 3)


In [ ]:
# top_words = 2000
# tokenizer = Tokenizer(num_words=top_words,split=' ')
# tokenizer.fit_on_texts(train_df['full_text'])
# list_tokenized_train = tokenizer.texts_to_sequences(train_df['full_text'])

# max_review_length = 200
# X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
# y_train = train_df['class']

top_words = 2000
tokenizer = Tokenizer(num_words=top_words, split=' ')
tokenizer.fit_on_texts(train_df['text_clean'])
list_tokenized_train = tokenizer.texts_to_sequences(train_df['text_clean'])

max_review_length = 200
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = train_df['class']

In [ ]:
# embedding_vecor_length = 32
# model = Sequential()
# model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
# model.add(LSTM(100))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=3)

embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words + 1, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))  # Update to 3 units and softmax activation
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_31 (Embedding)    (None, 200, 32)           64032     
                                                                 
 lstm_31 (LSTM)              (None, 100)               53200     
                                                                 
 dense_31 (Dense)            (None, 3)                 303       
                                                                 
Total params: 117,535
Trainable params: 117,535
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split=0.2)

Epoch 1/30
14/14 [==============================] - 9s 409ms/step - loss: 0.9642 - accuracy: 0.6505 - val_loss: 0.7983 - val_accuracy: 0.7130
Epoch 2/30
14/14 [==============================] - 8s 572ms/step - loss: 0.8380 - accuracy: 0.6840 - val_loss: 0.7864 - val_accuracy: 0.7130
Epoch 3/30
14/14 [==============================] - 5s 332ms/step - loss: 0.8034 - accuracy: 0.6840 - val_loss: 0.7490 - val_accuracy: 0.7130
Epoch 4/30
14/14 [==============================] - 6s 442ms/step - loss: 0.7581 - accuracy: 0.6887 - val_loss: 0.7032 - val_accuracy: 0.7130
Epoch 5/30
14/14 [==============================] - 5s 344ms/step - loss: 0.6808 - accuracy: 0.7350 - val_loss: 0.6471 - val_accuracy: 0.7500
Epoch 6/30
14/14 [==============================] - 5s 361ms/step - loss: 0.5604 - accuracy: 0.7697 - val_loss: 0.6360 - val_accuracy: 0.7407
Epoch 7/30
14/14 [==============================] - 7s 497ms/step - loss: 0.4531 - accuracy: 0.8021 - val_loss: 0.6543 - val_accuracy: 0.7407
Epoch 

In [ ]:
from sklearn.metrics import classification_report
list_tokenized_test = tokenizer.texts_to_sequences(test_df['text_clean'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)

y_test = test_df['class']
prediction = model.predict(X_test)
y_pred = np.argmax(prediction, axis=1)

classification_result = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_result)

4/4 [==============================] - 2s 230ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.86      0.79        80
           1       0.15      0.14      0.15        14
           2       0.33      0.15      0.21        26

    accuracy                           0.62       120
   macro avg       0.40      0.39      0.38       120
weighted avg       0.57      0.62      0.59       120



In [ ]:
test_df['predicted_sentiment'].value_counts()

0    87
2    24
1     9
Name: predicted_sentiment, dtype: int64

In [ ]:
test_df['class'].value_counts()

0    80
2    26
1    14
Name: class, dtype: int64

In [ ]:
test_df

,no,full_text,class,predicted_sentiment
1178,1178,transjakarta min gimana sihh aku baru turun an...,2,2
865,865,jaklingko lebih parah,1,0
101,101,jkt menarik mrtj jg hrs pilih from to stasiun ...,0,2
439,439,min tanya dong kalau terdaftar dtks bukan anak...,0,0
58,58,jaklingko,0,0
...,...,...,...,...
1037,1037,beli kartu flazz yg gampang mana pake jaklingk...,2,0
244,244,kalau jaklingko aku gapunya kartunya pake flaz...,0,0
158,158,jkt yaa banyak pilihan sih di jkt busway mrt k...,0,0
745,745,pagi saya naik jaklingko no kwk buaran menuj...,0,2


In [ ]:
# Membuat prediksi pada data pelatihan
test_predictions = model.predict(X_test)
test_pred_labels = np.argmax(test_predictions, axis=1)

# Menambahkan kolom prediksi pada train_df
test_df['predicted_sentiment'] = test_pred_labels

4/4 [==============================] - 0s 56ms/step


In [ ]:
test_df

,no,full_text,class,predicted_sentiment
533,533,kenalan sama aplikasi jaklingko yukk aplikasi ...,0,0
903,903,angkot jaklingko tuu anjir sering,0,0
1003,1003,tahun kemarin sempat tron kaya jaklingko baru ...,0,0
1069,1069,selamat pagi kak kartu jaklingko terbitan bank...,0,0
413,413,min pintu selatan st tanah abang yg akses jakl...,2,2
...,...,...,...,...
1044,1044,maraknya masyarakat menjadikan jaklingko sbg a...,0,0
1095,1095,gabisa langsung bayar pake qris qr code apalah...,2,0
1130,1130,iya makanya bilang kasihan bukan pemegang kart...,2,0
860,860,jaklingko taman kota budi luhur,0,0


In [ ]:
from tensorflow.keras.utils import to_categorical

max_akurasi = 0
max_f1 = 0
for i in range(1,20):
  df = df1

  train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42)

  top_words = 2000
  tokenizer = Tokenizer(num_words=top_words, split=' ')
  tokenizer.fit_on_texts(train_df['full_text'])
  list_tokenized_train = tokenizer.texts_to_sequences(train_df['full_text'])

  max_review_length = 200
  X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
  y_train = train_df['class']

  y_train = to_categorical(y_train, num_classes=3)

  embedding_vector_length = 32
  model = Sequential()
  model.add(Embedding(top_words + 1, embedding_vector_length, input_length=max_review_length))
  model.add(LSTM(100))
  model.add(Dense(3, activation='softmax'))  # Update to 3 units and softmax activation
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()

  model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

  list_tokenized_test = tokenizer.texts_to_sequences(test_df['full_text'])
  X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)

  y_test = test_df['class']
  prediction = model.predict(X_test)
  y_pred = np.argmax(prediction, axis=1)

  akurasi = accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred, average='weighted')
  print("Accuracy of the model:", akurasi)
  print("Confusion matrix:")
  print(confusion_matrix(y_test, y_pred))
  print("F1-score:", f1)

  if akurasi > max_akurasi:
    max = akurasi
  if akurasi > max:
    max = akurasi

print("akurasi: ", max_akurasi)
print("f1: ", max_f1)

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 200, 32)           64032     
                                                                 
 lstm_16 (LSTM)              (None, 100)               53200     
                                                                 
 dense_16 (Dense)            (None, 3)                 303       
                                                                 
Total params: 117,535
Trainable params: 117,535
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
12/12 [==============================] - 9s 582ms/step - loss: 1.0282 - accuracy: 0.6562 - val_loss: 0.8459 - val_accuracy: 0.6979
Epoch 2/20
12/12 [==============================] - 9s 713ms/step - loss: 0.8727 - accuracy: 0.6888 - val_loss: 0.8196 - val_accuracy: 0.6979
Epoch 3/20
12/12 [=========

KeyboardInterrupt: ignored

In [ ]:
print(max)

0.6333333333333333


In [ ]:
# banyak = 0
# value_counts = test_df['class'].value_counts().min()
# hapus0 = []
# hapus1 = []

# for i in range(0,len(test_df)):
#   row = test_df.iloc[i]
#   if row['class'] == 0:
#     hapus0.append(i)

# test_df = test_df.drop(index=hapus0).reset_index(drop=True)

# banyak = 0

# for i in range(0,len(test_df)):
#   row = test_df.iloc[i]
#   if row['class'] == 2:
#     banyak = banyak + 1
#   if banyak >= value_counts:
#     if row['class'] == 2:
#       hapus1.append(i)

# test_df = test_df.drop(index=hapus1).reset_index(drop=True)